In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os; os.chdir('C:\\work\\dev\\python\\progs\\scraper\\ufc\\')

In [3]:
from bs4 import BeautifulSoup
from datetime import datetime

import numpy as np
import pandas as pd
import re

In [4]:
import requests

In [5]:
from omegaconf import OmegaConf

conf = OmegaConf.load('params.yaml')

# fights coefs

In [8]:
url = 'https://leon.ru/api-2/betline/events/all?ctag=ru-RU&league_id=1970324837031381&hideClosed=true&flags=reg,urlv2,mm2,rrc,nodup'
# url = 'https://leon.ru/api-2/betline/events/all?ctag=ru-RU&region_id=1970324837031381&hideClosed=true&flags=reg,urlv2,mm2,rrc,nodup'
headers = {
    "accept": "*/*",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "priority": "u=1, i",
    "referer": "https://leon.ru/bets/mma/1970324836974960-ufc",
    "sec-ch-ua": '"Chromium";v="136", "Google Chrome";v="136", "Not.A/Brand";v="99"',
    "sec-ch-ua-mobile": "?1",
    "sec-ch-ua-platform": '"Android"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Mobile Safari/537.36",
    "x-app-browser": "chrome",
    "x-app-env": "prod",
    "x-app-language": "ru_RU",
    "x-app-layout": "phone",
    "x-app-modernity": "legacy",
    "x-app-os": "android",
    "x-app-platform": "web",
    "x-app-rendering": "csr",
    "x-app-skin": "default",
    "x-app-theme": "LIGHT",
    "x-app-version": "6.113.2",
    "x-requested-uri": "/bets/mma/1970324836974960-ufc"
}

cookies = {
    "__ddg1_": "aKv0JIzXhx2QEKciyhtm",
    "ABTestSeed": "37",
    "wm": "62398954",
    "click_id": "d75043555d8b576b57d00d5b",
    "ipfrom": "185.222.238.69",
    "referer": "https://bookmaker-ratings.ru/",
    "x-app-language": "ru_RU",
    "firstTheme": "LIGHT",
    "_ga": "GA1.1.803396759.1744464327",
    "_ym_uid": "1744464328338270285",
    "_ym_d": "1744464328",
    "qtag_rfrr": "null-null",
    "__ddg9_": "109.252.121.163",
    "_ym_isad": "2",
    "_ga_THNYNGV4VN": "GS2.1.s1748668513$o9$g1$t1748668821$j43$l0$h0",
    "__ddg10_": "1748668887",
    "__ddg8_": "dIW8Z0YF7bfSeQGV"
}

response = requests.get(url, headers=headers, cookies=cookies)

if response.status_code == 200:
    print("Request successful!")
    res_js = response.json()


Request successful!


In [9]:
event_date = datetime.fromtimestamp(res_js['events'][0]['kickoff']/1000)
event_date = event_date.strftime('%d.%m.%Y')
event_name = res_js['events'][0]['league']['name']
load_date = datetime.now().strftime('%Y-%m-%d')

In [10]:
res_l = []
for i in range(len(res_js['events'])):
    s = []
    fight = res_js['events'][i]
    # fighters = [it['name'] for it in res_js['events'][i]['competitors']]
    fighters = [it['name'] for it in fight['competitors'] if it['homeAway']=='HOME'] + [it['name'] for it in fight['competitors'] if it['homeAway']=='AWAY']
    # coefs = [it['price'] for it in res_js['events'][i]['markets'][0]['runners']]
    coefs = [it['price'] for it in fight['markets'][0]['runners'] if 'HOME' in it['tags']] + [it['price'] for it in fight['markets'][0]['runners'] if 'AWAY' in it['tags']]

    s.extend(fighters)
    s.extend(coefs)
    s.extend([event_date])
    s.extend([load_date])
    s.extend([event_name])
    s.extend(['leon.ru'])
    
    
    res_l.append(s)



fighter_coef_df = pd.DataFrame(res_l, columns=['fighter1', 'fighter2', 'coef1', 'coef2', 'date', 'load_date', 'name', 'source'])
fighter_coef_df

,fighter1,fighter2,coef1,coef2,date,load_date,name,source
0,Джейми-Лин Хорт,Тереза Бледа,2.38,1.56,14.12.2025,2025-12-07,UFC Fight Night - Ройвал - Капе,leon.ru
1,Сесар Алмейда,Цезары Олексейчук,2.37,1.57,14.12.2025,2025-12-07,UFC Fight Night - Ройвал - Капе,leon.ru
2,Мелкисаэль Коста,Морган Шарьер,1.80,1.98,14.12.2025,2025-12-07,UFC Fight Night - Ройвал - Капе,leon.ru
3,Мелисса Кроден,Луана Сантос,2.33,1.59,14.12.2025,2025-12-07,UFC Fight Night - Ройвал - Капе,leon.ru
4,Шон Шараф,Стивен Асплунд,3.20,1.33,14.12.2025,2025-12-07,UFC Fight Night - Ройвал - Капе,leon.ru
5,Аманда Лемос,Джиллиан Робертсон,2.33,1.59,14.12.2025,2025-12-07,UFC Fight Night - Ройвал - Капе,leon.ru
6,Йоандерсон Брито,Мелсик Багдасарян,1.45,2.70,14.12.2025,2025-12-07,UFC Fight Night - Ройвал - Капе,leon.ru
7,Нил Магни,Ярослав Амосов,3.90,1.24,14.12.2025,2025-12-07,UFC Fight Night - Ройвал - Капе,leon.ru
8,Кеннеди Нзечукву,Маркус Бучеча,1.61,2.28,14.12.2025,2025-12-07,UFC Fight Night - Ройвал - Капе,leon.ru
9,Жига Чикадже,Кевин Вальехос,2.80,1.42,14.12.2025,2025-12-07,UFC Fight Night - Ройвал - Капе,leon.ru


In [11]:
df_prev = pd.read_excel(conf['inference']['coef_fn'])


In [12]:
df_all = pd.concat([df_prev, fighter_coef_df], ignore_index=True)
df_all['coef1'] = df_all['coef1'].astype(float)

df_all['coef2'] = df_all['coef2'].astype(float)
# нули и пустоты удалить потом

df_all.tail()

,fighter1,fighter2,coef1,coef2,date,name,load_date,source
1680,Йоандерсон Брито,Мелсик Багдасарян,1.45,2.70,14.12.2025,UFC Fight Night - Ройвал - Капе,2025-12-07,leon.ru
1681,Нил Магни,Ярослав Амосов,3.90,1.24,14.12.2025,UFC Fight Night - Ройвал - Капе,2025-12-07,leon.ru
1682,Кеннеди Нзечукву,Маркус Бучеча,1.61,2.28,14.12.2025,UFC Fight Night - Ройвал - Капе,2025-12-07,leon.ru
1683,Жига Чикадже,Кевин Вальехос,2.80,1.42,14.12.2025,UFC Fight Night - Ройвал - Капе,2025-12-07,leon.ru
1684,Брэндон Ройвал,Манель Капе,2.90,1.39,14.12.2025,UFC Fight Night - Ройвал - Капе,2025-12-07,leon.ru


In [14]:
df_all.to_excel(conf['inference']['coef_fn'], index=False)

# UFC rankings

In [26]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

PROXY = "147.45.227.230:4080"          # <host>:<port>

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument(f'--proxy-server=http://{PROXY}')   # http

service = Service(r'c:\work\dev\python\progs\utils\chromedriver.exe')
browser = webdriver.Chrome(service=service, options=chrome_options)

In [27]:
browser.get('https://ufc.ru/rankings')
# browser.implicitly_wait(2)
html = browser.page_source


In [28]:
bsObj = BeautifulSoup(html, 'lxml')

weights = bsObj.findAll('tbody')
info_headers = bsObj.findAll('div', {'class':'info'})


In [29]:
fin_l = []
for i in range(len(info_headers)):
    division_name = info_headers[i].h4.get_text()
    slice_date = datetime.now().strftime('%Y-%m-%d')

    pattern1 = r'\n(\d+)\s+\n([^\n]+)\s+\nRank (increased|decreased) by (\d+)\s+'
    pattern2 = r'\n(\d+)\s+\n([^\n]+)\s+\n[^R]+\n'
    
    
    weight_strs = [it.get_text() for it in weights[i].findAll('tr')]
    
    main_name = info_headers[i].h5.get_text()

    if not 'вне' in division_name.lower():
        fin_l.append((main_name, 0, None, np.nan, division_name, slice_date))
    
    for text in weight_strs:
        match1 = re.search(pattern1, text)
        match2 = re.search(pattern2, text)
        if match1:
            rank = match1.group(1)
            name = match1.group(2)      # 'Илия Топурия'
            direction = match1.group(3)    # 'increased'
            amount = match1.group(4)    # '2'
            fin_l.append((name, rank, direction, amount, division_name, slice_date))
        elif match2:
            rank = match2.group(1)
            name = match2.group(2)
            fin_l.append((name, rank, None, np.nan, division_name, slice_date))
        else:
            fin_l.append((None, np.nan, None, np.nan, division_name, slice_date))

rank_df = pd.DataFrame(fin_l, columns=['name', 'rank', 'direction', 'val', 'division_name', 'date'])

## проверки

In [30]:
assert rank_df.isna().mean().loc[lambda x: x>0].index.tolist()==['direction', 'val'], 'Состав null полей поменялся'

In [31]:
assert ((rank_df==0).mean().loc[lambda x: x>0]).between(0.05, 0.06).iloc[0]==True, 'Пропорция нулевых рангов изменилась'

In [32]:
assert (rank_df=='').mean().loc[lambda x: x>0].shape[0]==0, 'Появились пустые строки'

## сохранение

In [33]:
rank_old_df = pd.read_csv(conf['inference']['rankings_fn'])
rank_df = pd.concat([rank_old_df, rank_df], ignore_index=True, axis=0)


In [34]:
import shutil

shutil.move(conf['inference']['rankings_fn'], f'data/arch/rank_df_{slice_date}.csv')
rank_df.to_csv(conf['inference']['rankings_fn'], index=False)

In [35]:
rank_df[(rank_df.division_name=='Вне зависимости от категорий Top Rank')&(rank_df['date']==rank_df['date'].max())].head(10)

,name,rank,direction,val,division_name,date
3914,Ислам Махачев,1,None,NaN,Вне зависимости от категорий Top Rank,2025-12-09
3915,Илия Топурия,2,None,NaN,Вне зависимости от категорий Top Rank,2025-12-09
3916,Хамзат Чимаев,3,increased,1,Вне зависимости от категорий Top Rank,2025-12-09
3917,Алекс Перейра,4,increased,2,Вне зависимости от категорий Top Rank,2025-12-09
3918,Александр Волкановски,5,increased,2,Вне зависимости от категорий Top Rank,2025-12-09
3919,Пётр Ян,6,None,NaN,Вне зависимости от категорий Top Rank,2025-12-09
3920,Том Аспинэлл,7,increased,1,Вне зависимости от категорий Top Rank,2025-12-09
3921,Мераб Двалишвили,8,decreased,5,Вне зависимости от категорий Top Rank,2025-12-09
3922,Александр Пантожа,9,decreased,4,Вне зависимости от категорий Top Rank,2025-12-09
3923,Макс Холлоуэй,10,None,NaN,Вне зависимости от категорий Top Rank,2025-12-09


# Topology rankings

In [36]:
from datetime import datetime

In [33]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

PROXY = "147.45.227.230:4080"          # <host>:<port>

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument(f'--proxy-server=http://{PROXY}')   # http

service = Service(r'c:\work\dev\python\progs\utils\chromedriver.exe')
browser = webdriver.Chrome(service=service, options=chrome_options)

In [37]:
weights = {'heavyweight': 'https://www.tapology.com/rankings/current-top-ten-heavyweight-mma-fighters-265-pounds',
           'light-heavyweight':'https://www.tapology.com/rankings/current-top-ten-light-heavyweight-mma-fighters-205-pounds',
            'middleweight':'https://www.tapology.com/rankings/current-top-ten-middleweight-mma-fighters-185-pounds',
            'welterweight':'https://www.tapology.com/rankings/current-top-ten-welterweight-mma-fighters-170-pounds',
            'lightweight':'https://www.tapology.com/rankings/current-top-ten-lightweight-mma-fighters-155-pounds',
            'featherweight':'https://www.tapology.com/rankings/current-top-ten-featherweight-mma-fighters-145-pounds',
            'bantamweight':'https://www.tapology.com/rankings/current-top-ten-bantamweight-mma-fighters-135-pounds',
            'flyweight':'https://www.tapology.com/rankings/35-top-flyweight-mma-fighters',
            'women-bantamweight':'https://www.tapology.com/rankings/1261-top-women-bantamweight-fighters',
            'women-flyweight':'https://www.tapology.com/rankings/1262-top-women-flyweight-fighters',
            'women-strawweight':'https://www.tapology.com/rankings/1263-top-women-strawweight-fighters',
            'women-atomweight':'https://www.tapology.com/rankings/1264-top-women-atomweight-fighters',
            'women-featherweight':'https://www.tapology.com/rankings/1265-top-women-featherweight-fighters'
          }

In [38]:
%%time
N = 2
fighters = []

slice_date = datetime.now().strftime('%Y-%m-%d')
for weight in weights.keys():
    fighters_page_d = {}
    fighters_page_d['division_name'] = weight
    fighters_page_d['date'] = slice_date
    
    for page in range(1, N+1):
        browser.get(f'{weights[weight]}?page={page}&ranking=10')
        # browser.implicitly_wait(0.1)
        html = browser.page_source

        bsObj = BeautifulSoup(html, 'lxml')
        
        lis = bsObj.findAll('li', {'class':'rankingItemsItem'})
        for i, info in enumerate(lis):
            
            fighters_page_d['rank'] = info.find('p').get_text().strip()
            fighters_page_d['name'] = info.find('h1').get_text().strip()
            
            fighters.append(fighters_page_d.copy())
        

CPU times: total: 4.59 s
Wall time: 43.7 s


In [39]:
rank_big_df = pd.DataFrame(fighters)

In [43]:
rank_big_df

,division_name,date,rank,name
0,heavyweight,2025-12-09,1,Tom Aspinall
1,heavyweight,2025-12-09,2,"Alexander ""Drago"" Volkov"
2,heavyweight,2025-12-09,3,"Ciryl ""Bon Gamin"" Gane"
3,heavyweight,2025-12-09,4,Sergei Pavlovich
4,heavyweight,2025-12-09,5,"Curtis ""Razor"" Blaydes"
...,...,...,...,...
1291,women-featherweight,2025-12-09,93,"Marilia ""The Tigress"" Morais"
1292,women-featherweight,2025-12-09,92,Charlotte McIntyre
1293,women-featherweight,2025-12-09,94,"Alana ""Lady Feral"" McLaughlin"
1294,women-featherweight,2025-12-09,95,Fiona Muxlow


## проверки

In [45]:
assert rank_big_df.loc[lambda x: x[['name', 'division_name']].duplicated(keep=False)].sort_values(by='name').shape[0] == 0, 'В рамках одного веса несколько записей о бойце' 

In [46]:
assert len(rank_big_df['division_name'].unique())==13, 'Количество категорий поменялось'

In [47]:
assert rank_big_df.isna().mean().loc[lambda x: x>0].index.tolist()==[], 'Состав null полей поменялся'

In [48]:
assert ((rank_big_df==0).mean().loc[lambda x: x>0]).shape[0]==0, 'Нулевые ранги появились откуда-то'

In [49]:
assert (rank_big_df=='').mean().loc[lambda x: x>0].shape[0]==0, 'Появились пустые строки'

## сохранение

In [50]:
rank_big_old_df = pd.read_csv('data/inference/rank_big_df.csv', dtype='str')


In [51]:
rank_big_df = pd.concat([rank_big_old_df, rank_big_df], ignore_index=True, axis=0)


In [52]:
import shutil

shutil.move('data/inference/rank_big_df.csv', f'data/arch/rank_df_{slice_date}.csv')
rank_big_df.to_csv('data/inference/rank_big_df.csv', index=False)

In [53]:
rank_big_df.shape, rank_big_old_df.shape

((14189, 4), (12893, 4))

# Статистика бойцов

In [59]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

PROXY = "147.45.227.230:4080"          # <host>:<port>

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument(f'--proxy-server=http://{PROXY}')   # http

service = Service(r'c:\work\dev\python\progs\utils\chromedriver.exe')
browser = webdriver.Chrome(service=service, options=chrome_options)

In [54]:
slice_date = datetime.now().strftime('%Y-%m-%d')

In [55]:
%%time
chars = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

res = []

for i in range(len(chars)):
    url = f'http://ufcstats.com/statistics/fighters?char={chars[i]}&page=all'

    browser.get(url)
    browser.implicitly_wait(1)
    html = browser.page_source
    bsObj = BeautifulSoup(html, 'lxml')
    fighter_rows = bsObj.findAll('tr', {'class':'b-statistics__table-row'})
    
    vals = []
    for j, row in enumerate(fighter_rows):
        if j==0 and i==0:
            colnms = [it.get_text().strip() for it in row.findAll('th', {'class':'b-statistics__table-col'})][:-4]
        # j==1 empty
        elif j>1:
            link = [it.get('href') for it in row.findAll('a') if 'fighter-details' in it.get('href')][0]
            vals.append([it.get_text().strip() for it in row.findAll('td', {'class':'b-statistics__table-col'})][:-4] +[link])
            
        
    res.extend(vals)

CPU times: total: 4.2 s
Wall time: 34 s


In [56]:
# fighters_stat_df = pd.DataFrame(res, columns=colnms+['link']).assign(slice_date=slice_date, dob=None)
fighters_stat_df = pd.DataFrame(res, columns=colnms+['link']).assign(slice_date=slice_date)

## проверки

In [57]:
assert len(set(chars) - set([it.lower() for it in fighters_stat_df['Last'].str[0].unique()]) )==0, 'Будто не все скачалось'

In [58]:
assert [it for it in fighters_stat_df['Last'].str[0].unique() if it.islower()] == ['d', 'v'], 'Есть маленькие буквы не только d, v'

In [59]:
assert fighters_stat_df.isna().mean().loc[lambda x: x>0].shape[0]==0, 'Нет nan-ов'

In [60]:
assert fighters_stat_df[~((fighters_stat_df.First=='Tony') & (fighters_stat_df.Last=='Johnson'))].duplicated(subset=['First', 'Last', 'Nickname']).sum() == 0 , 'Есть дубли по бойцам'

In [61]:
assert (fighters_stat_df==0).sum().loc[lambda x: x>0].shape[0] == 0

In [62]:
(fighters_stat_df=='').mean().loc[lambda x: x>0]

First       0.003596
Nickname    0.445493
Stance      0.189930
dtype: float64

In [63]:
fighters_stat_df_s = fighters_stat_df.copy()

## добавление старого

In [64]:
fighters_stat_old_df = pd.read_csv('data/inference/fighters_stat_df.csv', keep_default_na=False)
fighters_stat_df = pd.concat([fighters_stat_old_df, fighters_stat_df], ignore_index=True, axis=0)


In [65]:
# удалили полные дубли
fighters_stat_df = fighters_stat_df[~fighters_stat_df.drop(columns=['slice_date', 'dob']).duplicated(keep='first')]

In [66]:
# Если True, тут интересно, так как дубли неполные, надо операции ниже тщательно посмотреть, как действуют на дублях
fighters_stat_df[(~((fighters_stat_df.First=='Tony') & (fighters_stat_df.Last=='Johnson')))&fighters_stat_df.duplicated(['First', 'Last', 'Nickname'], keep=False)].shape[0]==0

False

In [67]:
fighters_stat_df[(~((fighters_stat_df.First=='Tony') & (fighters_stat_df.Last=='Johnson')))
&fighters_stat_df.duplicated(['First', 'Last', 'Nickname'], keep=False)].sort_values(by=['First', 'Last', 'Nickname'])

,First,Last,Nickname,Ht.,Wt.,Reach,Stance,link,slice_date,dob
179,Alexander,Volkov,Drago,"6' 7""",250 lbs.,"80.0""",Orthodox,http://ufcstats.com/fighter-details/279566840a...,2025-07-12,"Oct 24, 1988"
8698,Alexander,Volkov,Drago,"6' 7""",261 lbs.,"80.0""",Orthodox,http://ufcstats.com/fighter-details/279566840a...,2025-12-09,NaN
209,Allen,Frye Jr.,,--,265 lbs.,--,,http://ufcstats.com/fighter-details/73a78713bd...,2025-10-09,"Apr 15, 1998"
5784,Allen,Frye Jr.,,--,265 lbs.,--,Orthodox,http://ufcstats.com/fighter-details/73a78713bd...,2025-12-09,NaN
222,Amanda,Ribas,,"5' 3""",115 lbs.,"66.0""",Orthodox,http://ufcstats.com/fighter-details/beecb672a2...,2025-07-12,"Aug 26, 1993"
...,...,...,...,...,...,...,...,...,...,...
4818,Yousri,Belgaroui,,"6' 6""",185 lbs.,"79.0""",Orthodox,http://ufcstats.com/fighter-details/7674b836ce...,2025-12-09,NaN
4438,Yuji,Ephoevi-Ga,The Chef,"5' 9""",155 lbs.,"75.0""",Switch,http://ufcstats.com/fighter-details/919fc4bc3e...,2025-07-12,"Oct 19, 1999"
5612,Yuji,Ephoevi-Ga,The Chef,"6' 0""",155 lbs.,"75.0""",Switch,http://ufcstats.com/fighter-details/919fc4bc3e...,2025-12-09,NaN
4494,Zhu,Kangjie,,"5' 6""",145 lbs.,"70.0""",Orthodox,http://ufcstats.com/fighter-details/6f4ec5aa3e...,2025-07-12,"Jan 16, 1996"


In [68]:
# ищем короткие строки (пустые, nan) будем из дублей оставлять более заполненные
# dob не рассматриваем, просто потом скачаем снова, чтобы не париться
fighters_stat_df['short_ndob'] = fighters_stat_df.drop(columns='dob').astype(str).apply(lambda x: (x.str.len()<5).sum(), axis=1)
fighters_stat_df['short_dob'] = fighters_stat_df.astype(str).apply(lambda x: (x.str.len()<5).sum(), axis=1)
# fighters_stat_df.groupby(['First', 'Last', 'Nickname']).apply(lambda df: df[df.dob.notna()])

fighters_stat_df = fighters_stat_df.groupby(['First', 'Last', 'Nickname'], as_index=False).apply(lambda df: df.sort_values(by=['short_ndob', 'short_dob']).iloc[0])\
                                    .drop(columns=['short_ndob', 'short_dob'])


In [69]:
fighters_stat_df.query('Nickname=="Bomb"')

,First,Last,Nickname,Ht.,Wt.,Reach,Stance,link,slice_date,dob
63,Adam,Livingston,Bomb,"6' 2""",155 lbs.,"71.0""",Orthodox,http://ufcstats.com/fighter-details/515890922f...,2025-10-09,"Aug 15, 2001"


### сравнение несовпадений с тем, что было

In [70]:
(fighters_stat_df['dob'].str.len()>10).sum()

3987

## сохранение


In [71]:
import shutil

shutil.move('data/inference/fighters_stat_df.csv', f'data/arch/fighters_stat_df_{slice_date}.csv')
fighters_stat_df.to_csv('data/inference/fighters_stat_df.csv', index=False)

## качаем dob

In [72]:
fighters_stat_df = pd.read_csv('data/inference/fighters_stat_df.csv', keep_default_na=False)


сконкатенируем со старым и для бойцов, у которых либо нет link-а в истории либо нет dob в истории делаем запрос 

In [73]:
# fighters_dob = fighters_stat_df.groupby(['First', 'Last', 'Nickname']).apply(lambda x: (x['link']=='').mean()==1 or (x['dob']=='').mean()==1 or x['link'].isna().mean()==1 or x['dob'].isna().mean()==1).index
fighters_dob = fighters_stat_df[(fighters_stat_df.link.str.contains('http')) & ((fighters_stat_df.dob=='') | (fighters_stat_df.dob.isna()) )].set_index(['First', 'Last', 'Nickname']).index

fighters_dob_df = fighters_stat_df.set_index(['First', 'Last', 'Nickname']).loc[fighters_dob]
fighters_dob_df.shape

(518, 7)

In [74]:
%%time

res = []
for i in range(1, len(fighters_dob_df)):
    
    d = {}
    browser.get(fighters_dob_df['link'].iloc[i])
    browser.implicitly_wait(0.1)
    html = browser.page_source
    bsObj = BeautifulSoup(html, 'lxml')
    rows = bsObj.findAll('li', {'class':'b-list__box-list-item b-list__box-list-item_type_block'})
    dobel = [it for it in rows if 'DOB' in it.get_text()][0]
    try:
        dob = re.search(r'([A-Za-z]{3})\s+\d{1,2},\s+(\d{4})', dobel.get_text()).group(0)
    except: 
        dob = None
    s = fighters_dob_df.iloc[i].name
    d['First'] = s[0]
    d['Last'] = s[1]
    d['Nickname'] = s[2]
    d['dob'] = dob
    res.append(d)
    if i==1000:
        break
    

CPU times: total: 6.62 s
Wall time: 5min 12s


In [75]:
fighters_stat_df = fighters_stat_df.merge(pd.DataFrame(res), on=['First', 'Last', 'Nickname'], how='left')\
                .assign(dob=lambda x: np.where(x['dob_y'].notna(), x['dob_y'], x['dob_x']))\
                .drop(columns=['dob_y', 'dob_x'])

In [76]:
fighters_stat_df.query('Nickname=="Bomb"')

,First,Last,Nickname,Ht.,Wt.,Reach,Stance,link,slice_date,dob
63,Adam,Livingston,Bomb,"6' 2""",155 lbs.,"71.0""",Orthodox,http://ufcstats.com/fighter-details/515890922f...,2025-10-09,"Aug 15, 2001"


## сохранение


In [77]:
import shutil

shutil.move('data/inference/fighters_stat_df.csv', f'data/arch/fighters_stat_df_{slice_date}.csv')
fighters_stat_df.to_csv('data/inference/fighters_stat_df.csv', index=False)

1. Height: 5' 7"
Рост: 5 футов 7 дюймов ≈ 170 см

2. Weight: 135 lbs.
Вес: 135 фунтов ≈ 61,2 кг

3. Reach: 68"
Размах рук: 68 дюймов ≈ 173 см

4. Stance: Orthodox
Стойка: Обычная (правша)

5. DOB: Jan 28, 1996
Дата рождения: 28 января 1996 года


In [84]:
browser.quit()

# Агрегации коэффициентов

In [6]:
import pandas as pd
from io import StringIO



In [7]:
df= pd.read_excel(conf['inference']['coef_fn'])

In [8]:
print(df.loc[df.name=='UFC Fight Night - Ройвал - Капе', ['fighter1', 'fighter2']].to_markdown(index=False))

| fighter1         | fighter2           |
|:-----------------|:-------------------|
| Джейми-Лин Хорт  | Тереза Бледа       |
| Сесар Алмейда    | Цезары Олексейчук  |
| Мелкисаэль Коста | Морган Шарьер      |
| Мелисса Кроден   | Луана Сантос       |
| Шон Шараф        | Стивен Асплунд     |
| Аманда Лемос     | Джиллиан Робертсон |
| Йоандерсон Брито | Мелсик Багдасарян  |
| Нил Магни        | Ярослав Амосов     |
| Кеннеди Нзечукву | Маркус Бучеча      |
| Жига Чикадже     | Кевин Вальехос     |
| Брэндон Ройвал   | Манель Капе        |


Найди в Интернете свежие коэффициенты букмекерских фирм России, Америки и Европы, Азии на поединки турнира "UFC Fight Night - Ройвал - Капе", включающего бои из markdown таблицы ниже. 
Структурируй ответ по шагам: 
- Первый шаг составь список сайтов и ссылок на данные
- Второй шаг проверь актуальность данных, перейдя по ссылкам, удостоверься в правильности коэффициентов и составь таблицу: ссылка, проверка пройдена (да/нет).
- Третий шаг составь таблицу из пар соперников для которых найдены коэффициенты и проверена валидность ссылок
- Четвертый шаг - для всех пары соперников из третьего шага, сформируй таблицу в формате csv с колонками имя первого бойца (название fighter1), имя второго бойца (fighter2), коэффициент на первого бойца (coef1), коэффициент на второго бойца (coef2), название источника (source) и ссылка их источника (link).
- Пятый, финальный шаг - сконкатенируй таблицы из четвертого шага и добавь текущую дату в формате %Y-%m-%d, как колонку load_date.

markdown таблица с боями:
| fighter1         | fighter2           |
|:-----------------|:-------------------|
| Джейми-Лин Хорт  | Тереза Бледа       |
| Сесар Алмейда    | Цезары Олексейчук  |
| Мелкисаэль Коста | Морган Шарьер      |
| Мелисса Кроден   | Луана Сантос       |
| Шон Шараф        | Стивен Асплунд     |
| Аманда Лемос     | Джиллиан Робертсон |
| Йоандерсон Брито | Мелсик Багдасарян  |
| Нил Магни        | Ярослав Амосов     |
| Кеннеди Нзечукву | Маркус Бучеча      |
| Жига Чикадже     | Кевин Вальехос     |
| Брэндон Ройвал   | Манель Капе        |

- load_date старая, ссылки не или туда старые даты входят
- коэффициенты сильно отличающиеся от других/
leon-а

## [ds](https://chat.deepseek.com/)

мало, но тщательнее

In [9]:
csv_like = '''
fighter1,fighter2,coef1,coef2,source,link,load_date
Брэндон Ройвал,Манель Капе,+220,-295,MMA Junkie,https://mmajunkie.usatoday.com/.../ufc-fight-night-brandon-royval-vs-manel-kape-start-time-lineup-odds/87672463007/,2025-12-09
Жига Чикадже,Кевин Вальехос,+200,-265,MMA Junkie,https://mmajunkie.usatoday.com/.../ufc-fight-night-brandon-royval-vs-manel-kape-start-time-lineup-odds/87672463007/,2025-12-09
Сесар Алмейда,Цезары Олексейчук,+125,-160,MMA Junkie,https://mmajunkie.usatoday.com/.../ufc-fight-night-brandon-royval-vs-manel-kape-start-time-lineup-odds/87672463007/,2025-12-09
Морган Шарьер,Мелкисаэль Коста,+105,-125,MMA Junkie,https://mmajunkie.usatoday.com/.../ufc-fight-night-brandon-royval-vs-manel-kape-start-time-lineup-odds/87672463007/,2025-12-09
Маркус Бучеча,Кеннеди Нзечукву,+120,-155,MMA Junkie,https://mmajunkie.usatoday.com/.../ufc-fight-night-brandon-royval-vs-manel-kape-start-time-lineup-odds/87672463007/,2025-12-09
Аманда Лемос,Джиллиан Робертсон,+140,-185,MMA Junkie,https://mmajunkie.usatoday.com/.../ufc-fight-night-brandon-royval-vs-manel-kape-start-time-lineup-odds/87672463007/,2025-12-09
Мелсик Багдасарян,Йоандерсон Брито,+195,-260,MMA Junkie,https://mmajunkie.usatoday.com/.../ufc-fight-night-brandon-royval-vs-manel-kape-start-time-lineup-odds/87672463007/,2025-12-09
Ярослав Амосов,Нил Магни,-390,+280,MMA Junkie,https://mmajunkie.usatoday.com/.../ufc-fight-night-brandon-royval-vs-manel-kape-start-time-lineup-odds/87672463007/,2025-12-09
Стивен Асплунд,Шон Шараф,-245,+185,MMA Junkie,https://mmajunkie.usatoday.com/.../ufc-fight-night-brandon-royval-vs-manel-kape-start-time-lineup-odds/87672463007/,2025-12-09
Мелисса Кроден,Луана Сантос,+125,-160,MMA Junkie,https://mmajunkie.usatoday.com/.../ufc-fight-night-brandon-royval-vs-manel-kape-start-time-lineup-odds/87672463007/,2025-12-09
Тереза Бледа,Джейми-Лин Хорт,-160,+125,MMA Junkie,https://mmajunkie.usatoday.com/.../ufc-fight-night-brandon-royval-vs-manel-kape-start-time-lineup-odds/87672463007/,2025-12-09
'''
ds_df = pd.read_csv(StringIO(csv_like), sep=',')
ds_df

,fighter1,fighter2,coef1,coef2,source,link,load_date
0,Брэндон Ройвал,Манель Капе,220,-295,MMA Junkie,https://mmajunkie.usatoday.com/.../ufc-fight-n...,2025-12-09
1,Жига Чикадже,Кевин Вальехос,200,-265,MMA Junkie,https://mmajunkie.usatoday.com/.../ufc-fight-n...,2025-12-09
2,Сесар Алмейда,Цезары Олексейчук,125,-160,MMA Junkie,https://mmajunkie.usatoday.com/.../ufc-fight-n...,2025-12-09
3,Морган Шарьер,Мелкисаэль Коста,105,-125,MMA Junkie,https://mmajunkie.usatoday.com/.../ufc-fight-n...,2025-12-09
4,Маркус Бучеча,Кеннеди Нзечукву,120,-155,MMA Junkie,https://mmajunkie.usatoday.com/.../ufc-fight-n...,2025-12-09
5,Аманда Лемос,Джиллиан Робертсон,140,-185,MMA Junkie,https://mmajunkie.usatoday.com/.../ufc-fight-n...,2025-12-09
6,Мелсик Багдасарян,Йоандерсон Брито,195,-260,MMA Junkie,https://mmajunkie.usatoday.com/.../ufc-fight-n...,2025-12-09
7,Ярослав Амосов,Нил Магни,-390,280,MMA Junkie,https://mmajunkie.usatoday.com/.../ufc-fight-n...,2025-12-09
8,Стивен Асплунд,Шон Шараф,-245,185,MMA Junkie,https://mmajunkie.usatoday.com/.../ufc-fight-n...,2025-12-09
9,Мелисса Кроден,Луана Сантос,125,-160,MMA Junkie,https://mmajunkie.usatoday.com/.../ufc-fight-n...,2025-12-09


## [mistral](https://chat.mistral.ai/chat)

конкретные и верные почти все

In [10]:
csv_like = '''
fighter1;fighter2;coef1;coef2;source;link;load_date
Брэндон Ройвал;Манель Капе;+250;-334;OddsShark;https://www.oddsshark.com/ufc/fight-night-kape-vs-royval-odds-picks-december-13-2025;2025-12-07
Жига Чикадже;Кевин Вальехос;;;UFC;https://www.ufc.com/event/ufc-fight-night-december-13-2025;2025-12-07
Аманда Лемос;Джиллиан Робертсон;;;MMAJunkie;https://mmajunkie.usatoday.com/story/sports/ufc/2025/12/08/ufc-fight-night-brandon-royval-vs-manel-kape-start-time-lineup-odds/87672463007/;2025-12-07
Йоандерсон Брито;Мелсик Багдасарян;;;MMAOddsBreaker;https://www.mmaoddsbreaker.com/fight-odds/opening-odds/160742-opening-betting-odds-for-ufc-vegas-112-royval-vs-kape/;2025-12-07
Нил Магни;Ярослав Амосов;;;Yahoo Sports;https://uk.sports.yahoo.com/news/ufc-fight-night-266-watch-185030929.html;2025-12-07
Кеннеди Нзечукву;Маркус Бучеча;;;UFC;https://www.ufc.com/event/ufc-fight-night-december-13-2025;2025-12-07
Мелкисаэль Коста;Морган Шарьер;;;MMAJunkie;https://mmajunkie.usatoday.com/story/sports/ufc/2025/12/08/ufc-fight-night-brandon-royval-vs-manel-kape-start-time-lineup-odds/87672463007/;2025-12-07
Мелисса Кроден;Луана Сантос;;;MMAOddsBreaker;https://www.mmaoddsbreaker.com/fight-odds/opening-odds/160742-opening-betting-odds-for-ufc-vegas-112-royval-vs-kape/;2025-12-07
Шон Шараф;Стивен Асплунд;;;Yahoo Sports;https://uk.sports.yahoo.com/news/ufc-fight-night-266-watch-185030929.html;2025-12-07
Сезар Алмейда;Цезары Олексейчук;;;UFC;https://www.ufc.com/event/ufc-fight-night-december-13-2025;2025-12-07
Джейми-Лин Хорт;Тереза Бледа;;;MMAJunkie;https://mmajunkie.usatoday.com/story/sports/ufc/2025/12/08/ufc-fight-night-brandon-royval-vs-manel-kape-start-time-lineup-odds/87672463007/;2025-12-07

'''
mist_df = pd.read_csv(StringIO(csv_like), sep=';')
mist_df

,fighter1,fighter2,coef1,coef2,source,link,load_date
0,Брэндон Ройвал,Манель Капе,250.0,-334.0,OddsShark,https://www.oddsshark.com/ufc/fight-night-kape...,2025-12-07
1,Жига Чикадже,Кевин Вальехос,NaN,NaN,UFC,https://www.ufc.com/event/ufc-fight-night-dece...,2025-12-07
2,Аманда Лемос,Джиллиан Робертсон,NaN,NaN,MMAJunkie,https://mmajunkie.usatoday.com/story/sports/uf...,2025-12-07
3,Йоандерсон Брито,Мелсик Багдасарян,NaN,NaN,MMAOddsBreaker,https://www.mmaoddsbreaker.com/fight-odds/open...,2025-12-07
4,Нил Магни,Ярослав Амосов,NaN,NaN,Yahoo Sports,https://uk.sports.yahoo.com/news/ufc-fight-nig...,2025-12-07
5,Кеннеди Нзечукву,Маркус Бучеча,NaN,NaN,UFC,https://www.ufc.com/event/ufc-fight-night-dece...,2025-12-07
6,Мелкисаэль Коста,Морган Шарьер,NaN,NaN,MMAJunkie,https://mmajunkie.usatoday.com/story/sports/uf...,2025-12-07
7,Мелисса Кроден,Луана Сантос,NaN,NaN,MMAOddsBreaker,https://www.mmaoddsbreaker.com/fight-odds/open...,2025-12-07
8,Шон Шараф,Стивен Асплунд,NaN,NaN,Yahoo Sports,https://uk.sports.yahoo.com/news/ufc-fight-nig...,2025-12-07
9,Сезар Алмейда,Цезары Олексейчук,NaN,NaN,UFC,https://www.ufc.com/event/ufc-fight-night-dece...,2025-12-07


## [алиса](https://alice.yandex.ru/)

In [11]:
csv_like = '''

fighter1,fighter2,coef1,coef2,source,link,load_date
Джейми-Лин Хорт,Тереза Бледа,1.75,2.05,Oddspedia,https://www.oddspedia.com/ufc/jamie-hampton-vs-teresa-bleda,2025-12-09
Сесар Алмейда,Цезары Олексейчук,1.60,2.25,Pinnacle,https://www.pinnacle.com/en/mma/ufc/almeida-vs-oleksiejuk,2025-12-09
Мелкисаэль Коста,Морган Шарьер,1.80,1.95,Bet365,https://www.bet365.com/#/AC/B1/C1/D13/E37516184/F2/,2025-12-09
Мелисса Кроден,Луана Сантос,1.90,1.88,1xBet,https://1xbet.com/line/MMA/101234-UFC-f234,2025-12-09
Шон Шараф,Стивен Асплунд,1.70,2.10,DraftKings,https://sportsbook.draftkings.com/event/ufc-fight-night-2025/fight5,2025-12-09
Аманда Лемос,Джиллиан Робертсон,1.65,2.20,Betfair,https://www.betfair.com/exchange/plus/ufc/fight-amanda-vs-gillian,2025-12-09
Йоандерсон Брито,Мелсик Багдасарян,1.85,1.92,Ladbrokes,https://www.ladbrokes.com/sports/ufc/fight-brito-vs-bagdasaryan,2025-12-09
Нил Магни,Ярослав Амосов,2.10,1.70,Pinnacle,https://www.pinnacle.com/en/mma/ufc/magny-vs-amosov,2025-12-09
Кеннеди Нзечукву,Маркус Бучеча,1.78,1.98,1xBet,https://1xbet.com/line/MMA/101234-UFC-f236,2025-12-09
Жига Чикадже,Кевин Вальехос,1.62,2.30,Oddspedia,https://www.oddspedia.com/ufc/chikadze-vs-vallie-flagg,2025-12-09
Брэндон Ройвал,Манель Капе,1.95,1.85,Dafabet,https://www.dafabet.com/ru/sports/mma/royval-vs-kape,2025-12-09


'''
alice_df = pd.read_csv(StringIO(csv_like), sep=',')
alice_df

,fighter1,fighter2,coef1,coef2,source,link,load_date
0,Джейми-Лин Хорт,Тереза Бледа,1.75,2.05,Oddspedia,https://www.oddspedia.com/ufc/jamie-hampton-vs...,2025-12-09
1,Сесар Алмейда,Цезары Олексейчук,1.60,2.25,Pinnacle,https://www.pinnacle.com/en/mma/ufc/almeida-vs...,2025-12-09
2,Мелкисаэль Коста,Морган Шарьер,1.80,1.95,Bet365,https://www.bet365.com/#/AC/B1/C1/D13/E3751618...,2025-12-09
3,Мелисса Кроден,Луана Сантос,1.90,1.88,1xBet,https://1xbet.com/line/MMA/101234-UFC-f234,2025-12-09
4,Шон Шараф,Стивен Асплунд,1.70,2.10,DraftKings,https://sportsbook.draftkings.com/event/ufc-fi...,2025-12-09
5,Аманда Лемос,Джиллиан Робертсон,1.65,2.20,Betfair,https://www.betfair.com/exchange/plus/ufc/figh...,2025-12-09
6,Йоандерсон Брито,Мелсик Багдасарян,1.85,1.92,Ladbrokes,https://www.ladbrokes.com/sports/ufc/fight-bri...,2025-12-09
7,Нил Магни,Ярослав Амосов,2.10,1.70,Pinnacle,https://www.pinnacle.com/en/mma/ufc/magny-vs-a...,2025-12-09
8,Кеннеди Нзечукву,Маркус Бучеча,1.78,1.98,1xBet,https://1xbet.com/line/MMA/101234-UFC-f236,2025-12-09
9,Жига Чикадже,Кевин Вальехос,1.62,2.30,Oddspedia,https://www.oddspedia.com/ufc/chikadze-vs-vall...,2025-12-09


## [ernie](https://ernie.baidu.com/)
[ernie alter](https://huggingface.co/spaces/baidu/ernie_x1_turbo_demo)
будто приводит от балды

In [12]:
csv_like='''

fighter1,fighter2,coef1,coef2,source,link,load_date
Брэндон Ройвал,Манель Капе,245,-313,BetMGM,https://www.betmgm.com/sports/ufc,2025-12-09
Брэндон Ройвал,Манель Капе,250,-334,bet365,https://www.bet365.com/sports/combat-sports,2025-12-09
'''

ernie_df = pd.read_csv(StringIO(csv_like), sep=',')
ernie_df

,fighter1,fighter2,coef1,coef2,source,link,load_date
0,Брэндон Ройвал,Манель Капе,245,-313,BetMGM,https://www.betmgm.com/sports/ufc,2025-12-09
1,Брэндон Ройвал,Манель Капе,250,-334,bet365,https://www.bet365.com/sports/combat-sports,2025-12-09


## [giga](https://giga.chat/gigachat/)

мало, но тщательнее

In [13]:
csv_like = '''
fighter1,fighter2,coef1,coef2,source,link,load_date
Брэндон Ройвал,Манель Капе,1.63,2.5,Fon.bet,https://fon.bet/sports/mix-fights,2025-12-09
'''
giga_df = pd.read_csv(StringIO(csv_like), sep=',')
giga_df

,fighter1,fighter2,coef1,coef2,source,link,load_date
0,Брэндон Ройвал,Манель Капе,1.63,2.5,Fon.bet,https://fon.bet/sports/mix-fights,2025-12-09


## [perplexity](https://www.perplexity.ai/search)

In [15]:
csv_like = '''
fighter1,fighter2,coef1,coef2,source,link,load_date
"Джейми-Лин Хорт","Тереза Бледа","2.26","1.62","BestFightOdds FanDuel","https://www.bestfightodds.com","2025-12-09"
"Сесар Алмейда","Цезары Олексейчук","2.52","1.96","BestFightOdds FanDuel","https://www.bestfightodds.com","2025-12-09"
"Мелкисаэль Коста","Морган Шарьер","1.87","1.87","BestFightOdds FanDuel","https://www.bestfightodds.com","2025-12-09"
"Мелисса Кроден","Луана Сантос","3.02","1.30","BestFightOdds FanDuel","https://www.bestfightodds.com","2025-12-09"
"Шон Шараф","Стивен Асплунд","2.80","1.35","BestFightOdds FanDuel","https://www.bestfightodds.com","2025-12-09"
"Аманда Лемос","Джиллиан Робертсон","2.42","1.84","BestFightOdds FanDuel","https://www.bestfightodds.com","2025-12-09"
"Йоандерсон Брито","Мелсик Багдасарян","1.40","2.96","BestFightOdds FanDuel","https://www.bestfightodds.com","2025-12-09"
"Нил Магни","Ярослав Амосов","3.80","1.39","BestFightOdds FanDuel","https://www.bestfightodds.com","2025-12-09"
"Кеннеди Нзечукву","Маркус Бучеча","1.64","2.22","BestFightOdds FanDuel","https://www.bestfightodds.com","2025-12-09"
"Жига Чикадже","Кевин Вальехос","3.20","1.34","BestFightOdds FanDuel","https://www.bestfightodds.com","2025-12-09"
"Брэндон Ройвал","Манель Капе","3.20","1.34","BestFightOdds FanDuel","https://www.bestfightodds.com","2025-12-09"
"Брэндон Ройвал","Манель Капе","4.40","1.24","Duelbits","https://verdictmma.com/news/betting-odds-brandon-royval-vs-manel-kape/1562","2025-12-09"

'''
perp_df = pd.read_csv(StringIO(csv_like), sep=',')
perp_df

,fighter1,fighter2,coef1,coef2,source,link,load_date
0,Джейми-Лин Хорт,Тереза Бледа,2.26,1.62,BestFightOdds FanDuel,https://www.bestfightodds.com,2025-12-09
1,Сесар Алмейда,Цезары Олексейчук,2.52,1.96,BestFightOdds FanDuel,https://www.bestfightodds.com,2025-12-09
2,Мелкисаэль Коста,Морган Шарьер,1.87,1.87,BestFightOdds FanDuel,https://www.bestfightodds.com,2025-12-09
3,Мелисса Кроден,Луана Сантос,3.02,1.30,BestFightOdds FanDuel,https://www.bestfightodds.com,2025-12-09
4,Шон Шараф,Стивен Асплунд,2.80,1.35,BestFightOdds FanDuel,https://www.bestfightodds.com,2025-12-09
5,Аманда Лемос,Джиллиан Робертсон,2.42,1.84,BestFightOdds FanDuel,https://www.bestfightodds.com,2025-12-09
6,Йоандерсон Брито,Мелсик Багдасарян,1.40,2.96,BestFightOdds FanDuel,https://www.bestfightodds.com,2025-12-09
7,Нил Магни,Ярослав Амосов,3.80,1.39,BestFightOdds FanDuel,https://www.bestfightodds.com,2025-12-09
8,Кеннеди Нзечукву,Маркус Бучеча,1.64,2.22,BestFightOdds FanDuel,https://www.bestfightodds.com,2025-12-09
9,Жига Чикадже,Кевин Вальехос,3.20,1.34,BestFightOdds FanDuel,https://www.bestfightodds.com,2025-12-09


## [qwen](https://chat.qwen.ai/)


In [16]:
csv_like = '''
fighter1,fighter2,coef1,coef2,source,link,load_date
Брэндон Ройвал,Манель Капе,3.50,1.33,Fight Odds,https://fightodds.info/ufc-royval-kape,2025-12-09
Жига Чикадже,Кевин Вальехос,3.40,1.36,Fight Odds,https://fightodds.info/ufc-chikadze-vallejos,2025-12-09
Джейми-Лин Хорт,Тереза Бледа,1.67,2.25,Oddschecker,https://oddschecker.com/mma/horth-bleda,2025-12-09
Сесар Алмейда,Цезары Олексейчук,2.42,1.59,Best Fight Odds,https://bestfightodds.com/almeida-oleksiejczuk,2025-12-09
Мелкисаэль Коста,Морган Шарьер,1.95,1.98,Best Fight Odds,https://bestfightodds.com/costa-charriere,2025-12-09
Аманда Лемос,Джиллиан Робертсон,2.50,1.57,Fight Odds,https://fightodds.info/ufc-lemos-robertson,2025-12-09
Нил Магни,Ярослав Амосов,3.25,1.25,Oddschecker,https://oddschecker.com/mma/magny-amosov,2025-12-09

'''
qwen_df = pd.read_csv(StringIO(csv_like), sep=',')
qwen_df

,fighter1,fighter2,coef1,coef2,source,link,load_date
0,Брэндон Ройвал,Манель Капе,3.50,1.33,Fight Odds,https://fightodds.info/ufc-royval-kape,2025-12-09
1,Жига Чикадже,Кевин Вальехос,3.40,1.36,Fight Odds,https://fightodds.info/ufc-chikadze-vallejos,2025-12-09
2,Джейми-Лин Хорт,Тереза Бледа,1.67,2.25,Oddschecker,https://oddschecker.com/mma/horth-bleda,2025-12-09
3,Сесар Алмейда,Цезары Олексейчук,2.42,1.59,Best Fight Odds,https://bestfightodds.com/almeida-oleksiejczuk,2025-12-09
4,Мелкисаэль Коста,Морган Шарьер,1.95,1.98,Best Fight Odds,https://bestfightodds.com/costa-charriere,2025-12-09
5,Аманда Лемос,Джиллиан Робертсон,2.50,1.57,Fight Odds,https://fightodds.info/ufc-lemos-robertson,2025-12-09
6,Нил Магни,Ярослав Амосов,3.25,1.25,Oddschecker,https://oddschecker.com/mma/magny-amosov,2025-12-09


## [llama](https://llama.online/chat)

есть только общие ссылки

In [17]:
csv_like = '''
fighter1,fighter2,coef1,coef2,source,link,load_date
Джейми-Лин Хорт,Тереза Бледа,1.85,2.10,Betway,https://www.betway.com,2024-04-27
Сесар Алмейда,Цезары Олексейчук,1.75,2.00,1xBet,https://1xbet.ru,2024-04-27
Мелкисаэль Коста,Морган Шарьер,1.80,2.00,FanDuel,https://www.fanduel.com,2024-04-27
Мелисса Кроден,Луана Сантос,1.90,1.95,BetMGM,https://www.betmgm.com,2024-04-27
Шон Шараф,Стивен Асплунд,1.85,2.05,Betway,https://www.betway.com,2024-04-27
Аманда Лемос,Джиллиан Робертсон,1.70,2.20,Bet365,https://www.bet365.com,2024-04-27
Йоандерсон Брито,Мелсик Багдасарян,1.80,2.05,FanDuel,https://www.fanduel.com,2024-04-27
Нил Магни,Ярослав Амосов,1.65,2.15,188Bet,https://188bet.com,2024-04-27
Кеннеди Нзечукву,Маркус Бучеча,1.75,2.00,Betasia,https://betasia.com,2024-04-27
Жига Чикадже,Кевин Вальехос,1.85,2.10,Pinnacle,https://www.pinnacle.com,2024-04-27
Брэндон Ройвал,Манель Капе,1.70,2.05,1xBet,https://1xbet.ru,2024-04-27
'''
llama_df = pd.read_csv(StringIO(csv_like), sep=',')
llama_df

,fighter1,fighter2,coef1,coef2,source,link,load_date
0,Джейми-Лин Хорт,Тереза Бледа,1.85,2.10,Betway,https://www.betway.com,2024-04-27
1,Сесар Алмейда,Цезары Олексейчук,1.75,2.00,1xBet,https://1xbet.ru,2024-04-27
2,Мелкисаэль Коста,Морган Шарьер,1.80,2.00,FanDuel,https://www.fanduel.com,2024-04-27
3,Мелисса Кроден,Луана Сантос,1.90,1.95,BetMGM,https://www.betmgm.com,2024-04-27
4,Шон Шараф,Стивен Асплунд,1.85,2.05,Betway,https://www.betway.com,2024-04-27
5,Аманда Лемос,Джиллиан Робертсон,1.70,2.20,Bet365,https://www.bet365.com,2024-04-27
6,Йоандерсон Брито,Мелсик Багдасарян,1.80,2.05,FanDuel,https://www.fanduel.com,2024-04-27
7,Нил Магни,Ярослав Амосов,1.65,2.15,188Bet,https://188bet.com,2024-04-27
8,Кеннеди Нзечукву,Маркус Бучеча,1.75,2.00,Betasia,https://betasia.com,2024-04-27
9,Жига Чикадже,Кевин Вальехос,1.85,2.10,Pinnacle,https://www.pinnacle.com,2024-04-27


## [grok](https://huggingface.co/spaces/llamameta/Grok-4-heavy-free)
[родная сссылка, пока не доступна для РФ](https://x.ai/grok)

ссылки не проверяются, может из-за vpn

In [18]:
csv_like = '''
fighter1,fighter2,coef1,coef2,source,link,load_date
Брэндон Ройвал,Манель Капе,3.40,1.33,DraftKings,https://sportsbook.draftkings.com/leagues/mma/ufc,2025-12-09
Брэндон Ройвал,Манель Капе,3.30,1.34,FanDuel,https://sportsbook.fanduel.com/mma/ufc,2025-12-09
Брэндон Ройвал,Манель Капе,3.40,1.33,BetMGM,https://sports.betmgm.com/en/sports/mma-13/betting/usa-1,2025-12-09
Брэндон Ройвал,Манель Капе,3.50,1.30,OddsShark,https://www.oddsshark.com/ufc/fight-night-kape-vs-royval-odds-picks-december-13-2025,2025-12-09
Брэндон Ройвал,Манель Капе,3.45,1.32,Covers,https://www.covers.com/sport/mma/ufc/card-ufc-fight-night-odds-591357,2025-12-09
Брэндон Ройвал,Манель Капе,3.40,1.33,MMAOddsBreaker,https://www.mmaoddsbreaker.com/fight-odds/opening-odds/160742-opening-betting-odds-for-ufc-vegas-112-royval-vs-kape/,2025-12-09
Брэндон Ройвал,Манель Капе,3.50,1.30,Winline,https://winline.ru/stavki/sport/mma/ufc/ufc_fight_night__royval_vs_kape,2025-12-09
Брэндон Ройвал,Манель Капе,3.60,1.28,Fonbet,https://fon.bet/sports/mix-fights,2025-12-09
Брэндон Ройвал,Манель Капе,3.45,1.31,Liga Stavok,https://www.ligastavok.ru/championships/ufc-fight-night-royval-kape-id-61234,2025-12-09
Брэндон Ройвал,Манель Капе,3.40,1.33,Championat,https://www.championat.com/bets/article-6274567-ufc-fight-night-roival-kape-stavki-i-koefficienty-na-boi.html,2025-12-09
Брэндон Ройвал,Манель Капе,3.35,1.34,Bet365,https://www.bet365.com/hub/en-gb/mma/ufc/ufc-fight-night-royval-vs-kape,2025-12-09
Брэндон Ройвал,Манель Капе,3.40,1.33,Stake,https://stake.com/sports/mma/ufc/ufc-fight-night-royval-vs-kape,2025-12-09
Аманда Лемос,Джиллиан Робертсон,2.10,1.77,DraftKings,https://sportsbook.draftkings.com/leagues/mma/ufc,2025-12-09
Аманда Лемос,Джиллиан Робертсон,2.15,1.74,BetMGM,https://sports.betmgm.com/en/sports/mma-13/betting/usa-1,2025-12-09
Аманда Лемос,Джиллиан Робертсон,2.05,1.80,OddsShark,https://www.oddsshark.com/ufc/fight-night-kape-vs-royval-odds-picks-december-13-2025,2025-12-09
Аманда Лемос,Джиллиан Робертсон,2.12,1.75,Covers,https://www.covers.com/sport/mma/ufc/card-ufc-fight-night-odds-591357,2025-12-09
Аманда Лемос,Джиллиан Робертсон,2.10,1.77,Winline,https://winline.ru/stavki/sport/mma/ufc/ufc_fight_night__royval_vs_kape,2025-12-09
Аманда Лемос,Джиллиан Робертсон,2.08,1.78,Bet365,https://www.bet365.com/hub/en-gb/mma/ufc/ufc-fight-night-royval-vs-kape,2025-12-09
Кеннеди Нзечукву,Маркус Бучеча,1.91,1.91,MMAOddsBreaker,https://www.mmaoddsbreaker.com/fight-odds/opening-odds/160742-opening-betting-odds-for-ufc-vegas-112-royval-vs-kape/,2025-12-09
Кеннеди Нзечукву,Маркус Бучеча,1.87,1.95,FanDuel,https://sportsbook.fanduel.com/mma/ufc,2025-12-09
Кеннеди Нзечукву,Маркус Бучеча,1.90,1.90,Fonbet,https://fon.bet/sports/mix-fights,2025-12-09
Кеннеди Нзечукву,Маркус Бучеча,1.92,1.89,Liga Stavok,https://www.ligastavok.ru/championships/ufc-fight-night-royval-kape-id-61234,2025-12-09
Жига Чикадже,Кевин Вальехос,1.67,2.20,OddsShark,https://www.oddsshark.com/ufc/fight-night-kape-vs-royval-odds-picks-december-13-2025,2025-12-09
Жига Чикадже,Кевин Вальехос,1.65,2.25,Covers,https://www.covers.com/sport/mma/ufc/card-ufc-fight-night-odds-591357,2025-12-09
Жига Чикадже,Кевин Вальехос,1.68,2.18,Championat,https://www.championat.com/bets/article-6274567-ufc-fight-night-roival-kape-stavki-i-koefficienty-na-boi.html,2025-12-09
Жига Чикадже,Кевин Вальехос,1.70,2.15,Stake,https://stake.com/sports/mma/ufc/ufc-fight-night-royval-vs-kape,2025-12-09
Йоандерсон Брито,Мелсик Багдасарян,2.30,1.67,DraftKings,https://sportsbook.draftkings.com/leagues/mma/ufc,2025-12-09
Йоандерсон Брито,Мелсик Багдасарян,2.25,1.70,BetMGM,https://sports.betmgm.com/en/sports/mma-13/betting/usa-1,2025-12-09
Йоандерсон Брито,Мелсик Багдасарян,2.35,1.65,Winline,https://winline.ru/stavki/sport/mma/ufc/ufc_fight_night__royval_vs_kape,2025-12-09
Нил Магни,Ярослав Амосов,1.83,2.00,MMAOddsBreaker,https://www.mmaoddsbreaker.com/fight-odds/opening-odds/160742-opening-betting-odds-for-ufc-vegas-112-royval-vs-kape/,2025-12-09
Нил Магни,Ярослав Амосов,1.80,2.05,FanDuel,https://sportsbook.fanduel.com/mma/ufc,2025-12-09
Нил Магни,Ярослав Амосов,1.85,1.95,Bet365,https://www.bet365.com/hub/en-gb/mma/ufc/ufc-fight-night-royval-vs-kape,2025-12-09

'''
grok_df = pd.read_csv(StringIO(csv_like), sep=',')
grok_df

,fighter1,fighter2,coef1,coef2,source,link,load_date
0,Брэндон Ройвал,Манель Капе,3.40,1.33,DraftKings,https://sportsbook.draftkings.com/leagues/mma/ufc,2025-12-09
1,Брэндон Ройвал,Манель Капе,3.30,1.34,FanDuel,https://sportsbook.fanduel.com/mma/ufc,2025-12-09
2,Брэндон Ройвал,Манель Капе,3.40,1.33,BetMGM,https://sports.betmgm.com/en/sports/mma-13/bet...,2025-12-09
3,Брэндон Ройвал,Манель Капе,3.50,1.30,OddsShark,https://www.oddsshark.com/ufc/fight-night-kape...,2025-12-09
4,Брэндон Ройвал,Манель Капе,3.45,1.32,Covers,https://www.covers.com/sport/mma/ufc/card-ufc-...,2025-12-09
5,Брэндон Ройвал,Манель Капе,3.40,1.33,MMAOddsBreaker,https://www.mmaoddsbreaker.com/fight-odds/open...,2025-12-09
6,Брэндон Ройвал,Манель Капе,3.50,1.30,Winline,https://winline.ru/stavki/sport/mma/ufc/ufc_fi...,2025-12-09
7,Брэндон Ройвал,Манель Капе,3.60,1.28,Fonbet,https://fon.bet/sports/mix-fights,2025-12-09
8,Брэндон Ройвал,Манель Капе,3.45,1.31,Liga Stavok,https://www.ligastavok.ru/championships/ufc-fi...,2025-12-09
9,Брэндон Ройвал,Манель Капе,3.40,1.33,Championat,https://www.championat.com/bets/article-627456...,2025-12-09


## chat gpt

In [19]:
csv_like = '''
fighter1,fighter2,coef1,coef2,source,link,load_date
"Джейми-Лин Хорт","Тереза Бледа","+121","-147","VerdictMMA / OddsChecker","https://www.oddschecker.com/ufc-mma/ufc/brandon-royval-v-manel-kape/winner",2025-12-09
"Сесар Алмейда","Цезары Олексейчук","2.65","1.57","Fonbet / OddsChecker","https://fonbet.com.cy/sports/mix-fights",2025-12-09
"Мелкисаэль Коста","Морган Шарьер","-110","-111","LowKick / OddsChecker","https://www.lowkickmma.com/brandon-royval-vs-manel-kape-fight-card-betting-odds-start-time/",2025-12-09
"Мелисса Кроден","Луана Сантос","+131","-161","VerdictMMA / DraftKings","https://verdictmma.com/news/betting-odds-brandon-royval-vs-manel-kape/",2025-12-09
"Шон Шараф","Стивен Асплунд","+180","-222","VerdictMMA / FightOdds.io","https://verdictmma.com/news/betting-odds-brandon-royval-vs-manel-kape/",2025-12-09
"Аманда Лемос","Джиллиан Робертсон","+137","-169","DraftKings / RotoWire","https://www.rotowire.com/betting/mma/fight/manel-kape-vs-brandon-royval-odds-2025-12-13-4826",2025-12-09
"Йоандерсон Брито","Мелcик Багдасарян","-256","+200","VerdictMMA / BestFightOdds","https://www.bestfightodds.com/",2025-12-09
"Нил Магни","Ярослав Амосов","+300","-400","VerdictMMA / RotoWire","https://verdictmma.com/news/betting-odds-brandon-royval-vs-manel-kape/",2025-12-09
"Кеннеди Нзечукву","Маркус Бучеча","-145","+119","LowKick / OddsChecker","https://www.lowkickmma.com/brandon-royval-vs-manel-kape-fight-card-betting-odds-start-time/",2025-12-09
"Жига Чикадже","Кевин Вальехос","+215","-278","FightOdds / OddsChecker","https://fightodds.io/",2025-12-09
"Брэндон Ройвал","Манель Капе","+240","-313","VerdictMMA / OddsShark / DraftKings","https://www.oddsshark.com/ufc/manel-kape-brandon-royval-odds-december-13-2025-2484489",2025-12-09

'''
chat_df = pd.read_csv(StringIO(csv_like), sep=',')
chat_df

,fighter1,fighter2,coef1,coef2,source,link,load_date
0,Джейми-Лин Хорт,Тереза Бледа,121.00,-147.00,VerdictMMA / OddsChecker,https://www.oddschecker.com/ufc-mma/ufc/brando...,2025-12-09
1,Сесар Алмейда,Цезары Олексейчук,2.65,1.57,Fonbet / OddsChecker,https://fonbet.com.cy/sports/mix-fights,2025-12-09
2,Мелкисаэль Коста,Морган Шарьер,-110.00,-111.00,LowKick / OddsChecker,https://www.lowkickmma.com/brandon-royval-vs-m...,2025-12-09
3,Мелисса Кроден,Луана Сантос,131.00,-161.00,VerdictMMA / DraftKings,https://verdictmma.com/news/betting-odds-brand...,2025-12-09
4,Шон Шараф,Стивен Асплунд,180.00,-222.00,VerdictMMA / FightOdds.io,https://verdictmma.com/news/betting-odds-brand...,2025-12-09
5,Аманда Лемос,Джиллиан Робертсон,137.00,-169.00,DraftKings / RotoWire,https://www.rotowire.com/betting/mma/fight/man...,2025-12-09
6,Йоандерсон Брито,Мелcик Багдасарян,-256.00,200.00,VerdictMMA / BestFightOdds,https://www.bestfightodds.com/,2025-12-09
7,Нил Магни,Ярослав Амосов,300.00,-400.00,VerdictMMA / RotoWire,https://verdictmma.com/news/betting-odds-brand...,2025-12-09
8,Кеннеди Нзечукву,Маркус Бучеча,-145.00,119.00,LowKick / OddsChecker,https://www.lowkickmma.com/brandon-royval-vs-m...,2025-12-09
9,Жига Чикадже,Кевин Вальехос,215.00,-278.00,FightOdds / OddsChecker,https://fightodds.io/,2025-12-09


## [kimi](https://www.kimi.com/chat)

In [20]:
csv_like = '''
fighter1,fighter2,coef1,coef2,source,link,load_date
Брэндон Ройвал,Манель Капе,-120,+100,ufc.com,https://www.ufc.com/event/ufc-fight-night-december-13-2025,2025-12-09
Жига Чикадзе,Кевин Вальехос,-278,+225,ufc.com,https://www.ufc.com/event/ufc-fight-night-december-13-2025,2025-12-09
Кеннеди Нзечукву,Маркус Бучеча,-225,+185,ufc.com,https://www.ufc.com/event/ufc-fight-night-december-13-2025,2025-12-09
Йоандерсон Брито,Мелсик Багдасарян,-150,+125,ufc.com,https://www.ufc.com/event/ufc-fight-night-december-13-2025,2025-12-09
Нил Магни,Ярослав Амосов,+138,-162,ufc.com,https://www.ufc.com/event/ufc-fight-night-december-13-2025,2025-12-09
Аманда Лемос,Джиллиан Робертсон,-185,+155,ufc.com,https://www.ufc.com/event/ufc-fight-night-december-13-2025,2025-12-09
Мелисса Кроден,Луана Сантос,+190,-230,ufc.com,https://www.ufc.com/event/ufc-fight-night-december-13-2025,2025-12-09
Шон Шараф,Стивен Асплунд,-215,+175,ufc.com,https://www.ufc.com/event/ufc-fight-night-december-13-2025,2025-12-09
Мелкизael Коста,Морган Шарьер,-135,+115,ufc.com,https://www.ufc.com/event/ufc-fight-night-december-13-2025,2025-12-09
Сесар Алмейда,Цезарий Олексейчук,-200,+170,ufc.com,https://www.ufc.com/event/ufc-fight-night-december-13-2025,2025-12-09
Джейми-Лин Хорт,Тереза Бледа,-110,-110,ufc.com,https://www.ufc.com/event/ufc-fight-night-december-13-2025,2025-12-09
'''
kimi_df = pd.read_csv(StringIO(csv_like), sep=',')
kimi_df

,fighter1,fighter2,coef1,coef2,source,link,load_date
0,Брэндон Ройвал,Манель Капе,-120,100,ufc.com,https://www.ufc.com/event/ufc-fight-night-dece...,2025-12-09
1,Жига Чикадзе,Кевин Вальехос,-278,225,ufc.com,https://www.ufc.com/event/ufc-fight-night-dece...,2025-12-09
2,Кеннеди Нзечукву,Маркус Бучеча,-225,185,ufc.com,https://www.ufc.com/event/ufc-fight-night-dece...,2025-12-09
3,Йоандерсон Брито,Мелсик Багдасарян,-150,125,ufc.com,https://www.ufc.com/event/ufc-fight-night-dece...,2025-12-09
4,Нил Магни,Ярослав Амосов,138,-162,ufc.com,https://www.ufc.com/event/ufc-fight-night-dece...,2025-12-09
5,Аманда Лемос,Джиллиан Робертсон,-185,155,ufc.com,https://www.ufc.com/event/ufc-fight-night-dece...,2025-12-09
6,Мелисса Кроден,Луана Сантос,190,-230,ufc.com,https://www.ufc.com/event/ufc-fight-night-dece...,2025-12-09
7,Шон Шараф,Стивен Асплунд,-215,175,ufc.com,https://www.ufc.com/event/ufc-fight-night-dece...,2025-12-09
8,Мелкизael Коста,Морган Шарьер,-135,115,ufc.com,https://www.ufc.com/event/ufc-fight-night-dece...,2025-12-09
9,Сесар Алмейда,Цезарий Олексейчук,-200,170,ufc.com,https://www.ufc.com/event/ufc-fight-night-dece...,2025-12-09


## Сохранение

In [21]:
from datetime import datetime
slice_date = datetime.now().strftime('%Y-%m-%d')

In [22]:
# df = kimi_df.assign(llm='kimi')

df = pd.concat([llama_df.assign(llm='llama'), mist_df.assign(llm='mistral'), grok_df.assign(llm='grok'), alice_df.assign(llm='alice'), 
                ernie_df.assign(llm='ernie'), giga_df.assign(llm='gigachat'), ds_df.assign(llm='deepseek'), 
                perp_df.assign(llm='perplexity'), qwen_df.assign(llm='qwen'), chat_df.assign(llm='chatgpt')], axis=0)\
        .assign(manual_load = slice_date)
df

,fighter1,fighter2,coef1,coef2,source,link,load_date,llm,manual_load
0,Джейми-Лин Хорт,Тереза Бледа,1.85,2.10,Betway,https://www.betway.com,2024-04-27,llama,2025-12-09
1,Сесар Алмейда,Цезары Олексейчук,1.75,2.00,1xBet,https://1xbet.ru,2024-04-27,llama,2025-12-09
2,Мелкисаэль Коста,Морган Шарьер,1.80,2.00,FanDuel,https://www.fanduel.com,2024-04-27,llama,2025-12-09
3,Мелисса Кроден,Луана Сантос,1.90,1.95,BetMGM,https://www.betmgm.com,2024-04-27,llama,2025-12-09
4,Шон Шараф,Стивен Асплунд,1.85,2.05,Betway,https://www.betway.com,2024-04-27,llama,2025-12-09
...,...,...,...,...,...,...,...,...,...
6,Йоандерсон Брито,Мелcик Багдасарян,-256.00,200.00,VerdictMMA / BestFightOdds,https://www.bestfightodds.com/,2025-12-09,chatgpt,2025-12-09
7,Нил Магни,Ярослав Амосов,300.00,-400.00,VerdictMMA / RotoWire,https://verdictmma.com/news/betting-odds-brand...,2025-12-09,chatgpt,2025-12-09
8,Кеннеди Нзечукву,Маркус Бучеча,-145.00,119.00,LowKick / OddsChecker,https://www.lowkickmma.com/brandon-royval-vs-m...,2025-12-09,chatgpt,2025-12-09
9,Жига Чикадже,Кевин Вальехос,215.00,-278.00,FightOdds / OddsChecker,https://fightodds.io/,2025-12-09,chatgpt,2025-12-09


In [23]:
import shutil

old_df = pd.read_csv(conf['inference']['llm_coef_fn'])

shutil.move(conf['inference']['llm_coef_fn'], f'data/arch/llm_coefs_{slice_date}.csv')



df = pd.concat([old_df, df], ignore_index=True)

In [24]:
df.shape, old_df.shape

((1236, 10), (1127, 10))

In [25]:
df.to_csv(conf['inference']['llm_coef_fn'], index=False)

## перевод форматов

In [84]:
df = pd.read_csv(conf['inference']['llm_coef_fn'], dtype='str')
df['coef1t'] = pd.to_numeric(df['coef1'], errors='coerce')
df['coef2t'] = pd.to_numeric(df['coef2'], errors='coerce')
df['not_num'] = df['coef1t'].isnull() | df['coef2t'].isnull()


In [ ]:
american_coef_sel = (df['coef1'].str.contains(r'\+|\-', regex=True, na=False)) | (df['coef2'].str.contains(r'\+|\-', regex=True, na=False))
# не типа not numeric и хотя бы один с нереалистичным остатком для американских коэффициентов
nreal_american_coef_sel = (df.apply(lambda x: float(x['coef1'])%1>0 or float(x['coef2'])%1>0 if not x['not_num'] else False, axis=1))
real_american_coef_sel = sel & (~nreal_american_coef_sel)&(~df['not_num']) 


df.loc[real_american_coef_sel, 'tr_coef1'] = df.loc[real_american_coef_sel, 'coef1'].astype(float).map(lambda x: x/100 + 1 if x>0 else 100/abs(x) + 1)
df.loc[real_american_coef_sel, 'tr_coef2'] = df.loc[real_american_coef_sel, 'coef2'].astype(float).map(lambda x: x/100 + 1 if x>0 else 100/abs(x) + 1)

df['coef1'] = df['coef1t'].mask(real_american_coef_sel, df['tr_coef1']).mask(sel & nreal_american_coef_sel, np.nan)
df['coef2'] = df['coef2t'].mask(real_american_coef_sel, df['tr_coef2']).mask(sel & nreal_american_coef_sel, np.nan)

# Словарь имен бойцов

In [13]:
import pandas as pd

from ruamel.yaml import YAML
import re
import numpy as np

conf = YAML().load(open('params.yaml'))

In [14]:
query_name = 'UFC Fight Night: Имавов – Борральо'

coef_df = pd.read_excel(conf['inference']['coef_fn'])
coef_df = coef_df.assign(event_day=lambda x:pd.to_datetime(x['date'], format='%d.%m.%Y'))
if not query_name:
    pred_df = coef_df[(coef_df['event_day']==coef_df['event_day'].max()) & (coef_df['load_date']==coef_df['load_date'].max())]
else:
    sel = (coef_df['name']==query_name)
    sel = sel & (coef_df.load_date == coef_df.loc[sel, 'load_date'].max())
    pred_df = coef_df.loc[sel]

## DeepSeek

In [35]:
fighters_l = pd.concat([pred_df[['fighter1']].rename(columns={'fighter1':'fighter'}), pred_df[['fighter2']].rename(columns={'fighter2':'fighter'})])\
    .drop_duplicates()['fighter'].tolist()
fighters_l

['Шона Бэннон',
 'Андреас Густафссон',
 'Роберт Брисжек',
 'Сэм Паттерсон',
 'Марцин Тубура',
 'Омар Си',
 'Уильям Гомис',
 'Гарри Хардвик',
 'Патрисиу Фрейри',
 'Рус Макки',
 'Боладжи Оки',
 'Модестас Букаускас',
 'Бенуа Сен-Дени',
 'Нассурдин Имавов',
 'Сэм Хьюз',
 'Ринат Фахретдинов',
 'Брэд Таварес',
 'Трей Уотерс',
 'Анте Делия',
 'Брендсон Рибейро',
 'Роберт Рухала',
 'Кауэ Фернандес',
 'Лосене Кейта',
 'Аксель Сола',
 'Мэйсон Джонс',
 'Пол Крэйг',
 'Маурисио Раффи',
 'Кайо Борральо']

Я тебе передам список имен бойцов ufc на русском языке, а ты верни мне для каждого бойца имена на английском, следи за сохранением порядка, в качестве разделителя используй "—",  не пытайся менять имена:



In [44]:
s = '''Шона Бэннон—Shauna Bannon
Андреас Густафссон—Andreas Gustafsson
Роберт Брисжек—Robert Bryczek
Сэм Паттерсон—Sam Patterson
Марцин Тубура—Marcin Tybura
Омар Си—Omar Sy
Уильям Гомис—William Gomis
Гарри Хардвик—Harry Hardwick
Патрисиу Фрейри—Patricio Freire
Рус Макки—Rhys McKee
Боладжи Оки—Bolaji Oki
Модестас Букаускас—Modestas Bukauskas
Бенуа Сен-Дени—Benoit Saint Denis
Нассурдин Имавов—Nassourdine Imavov
Сэм Хьюз—Sam Hughes
Ринат Фахретдинов—Rinat Fakhretdinov
Брэд Таварес—Brad Tavares
Трей Уотерс—Trey Waters
Анте Делия—Ante Delija
Брендсон Рибейро—Brendson Ribeiro
Роберт Рухала—Robert Ruchala
Кауэ Фернандес—Caio Fernandes
Лосене Кейта—Loseine Keita
Аксель Сола—Axel Sola
Мэйсон Джонс—Mason Jones
Пол Крэйг—Paul Craig
Маурисио Раффи—Mauricio Ruffy
Кайо Борральо—Caio Borralho'''

items = [it for it in s.split('\n') if len(it)>0]


In [45]:
import pandas as pd

vocab_df = pd.DataFrame([(it.split('—')[0].strip(), it.split('—')[1].strip()) for it in items], columns=['name_rus', 'name_en_ds'])
vocab_df.head(2)

,name_rus,name_en_ds
0,Шона Бэннон,Shauna Bannon
1,Андреас Густафссон,Andreas Gustafsson


In [46]:
from metaphone import doublemetaphone

vocab_df['code'] = vocab_df['name_en_ds'].map(lambda x: doublemetaphone(x)[0])

vocab_df['manual_verified']=False

In [47]:
vocab_old_df = pd.read_csv(conf['inference']['vocab_fn'])

In [48]:
# То, что есть в старом и прошло проверку по всем датасетам - удаляем
names_ser = vocab_old_df.loc[((vocab_old_df.name_rus.isin(vocab_df.name_rus) & (vocab_old_df[['fighters_name', 'fights_name', 'ranks_name']].isna().sum(axis=1)==0))), 'name_rus']
vocab_df = vocab_df[~vocab_df.name_rus.isin(names_ser)]

In [49]:
# Имеется в старом словаре, но полностью не заполнено
vocab_df[(vocab_df.name_rus.isin(vocab_old_df.name_rus)) & (~vocab_df.name_rus.isin(names_ser))]

,name_rus,name_en_ds,code,manual_verified


## ufcs stats names

In [14]:
from selenium import webdriver
browser = webdriver.Firefox()

In [15]:
browser.get('http://www.ufcstats.com/event-details/6e380a4d73ab4f0e')
browser.implicitly_wait(2)
html = browser.page_source


In [16]:
bsObj = BeautifulSoup(html, 'lxml')

fighters = bsObj.findAll('p', {'class':'b-fight-details__table-text'})

f_names = [tag.get_text().strip() for tag in fighters]
f_names = [it for it in f_names if it!='' and not 'Matchup' in it and not 'weight' in it]

site_df = pd.DataFrame({'name_en':f_names}).assign(code=lambda x: x['name_en'].map(lambda y: doublemetaphone(y)[0]))

In [50]:
vocab_df = vocab_df.merge(site_df, how='outer', on='code')


- если в соединении нет записи в fights, то будет и рус и name_en (name_en_ds), и code (code1) , как, видимо, в конце удалим запись, так как вряд ли в fights найдем
- если не будет записи на русском, надо сверху добавить рус и англ название (например, через deep seek)

### no rus

In [51]:
vocab_df[vocab_df['name_rus'].isna()]

,name_rus,name_en_ds,code,manual_verified,name_en


### no ufc

In [56]:
vocab_df[vocab_df['name_en'].isna()]

,name_rus,name_en_ds,code,manual_verified,name_en


In [57]:
vocab_df['name_en'] = vocab_df['name_en'].mask(vocab_df['name_en'].isna(), vocab_df['name_en_ds'])

## проверка по датасету бойцов

In [58]:
fighters_df = pd.read_csv(conf['inference']['fighters_fn'])
fighters_df['full_name'] = fighters_df['First'].str.cat(fighters_df['Last'], sep=' ', na_rep='')

In [59]:
fighters_part_df = fighters_df[['full_name']].drop_duplicates()

vocab_df = vocab_df.merge(fighters_part_df, how='left', left_on='name_en', right_on='full_name')\
        .assign(check_fighters=lambda x:np.where(x['full_name'].notna(), True, False))\
        .rename(columns={'full_name':'fighters_name'})



In [60]:
fighters_part_df['code'] = fighters_part_df['full_name'].map(lambda x: doublemetaphone(x)[0])

vocab_df = vocab_df.merge(fighters_part_df.groupby('code', as_index=False)['full_name'].agg(list).rename(columns={'full_name':'fighters_alt'}), on='code', how='left')

## проверка по датасету боев

In [61]:
from src.prep_funcs import prep_strikes_fn

fights_df = pd.read_csv(conf['inference']['fights_fn'])

C:\Users\admin\AppData\Local\Temp\ipykernel_12572\1519535284.py:3: DtypeWarning: Columns (205,206,207,208) have mixed types. Specify dtype option on import or set low_memory=False.
  fights_df = pd.read_csv(conf['inference']['fights_fn'])


In [62]:
fighters_from_fights = pd.concat([fights_df[['Fighter_left']].rename(columns={'Fighter_left':'name'}), 
fights_df[['Fighter_right']].rename(columns={'Fighter_right':'name'})], axis=0).drop_duplicates()

In [63]:
vocab_df = vocab_df.merge(fighters_from_fights, how='left', left_on='name_en', right_on='name')\
        .assign(check_fights=lambda x:np.where(x['name'].notna(), True, False))\
        .rename(columns={'name':'fights_name'})


In [64]:
fighters_from_fights['code'] = fighters_from_fights['name'].map(lambda x: doublemetaphone(x)[0])

vocab_df = vocab_df.merge(fighters_from_fights.groupby('code', as_index=False)['name'].agg(list).rename(columns={'name':'fights_alt'}), on='code', how='left')

## проверка по датасету рангов

In [65]:
rank_big_df = pd.read_csv(conf['inference']['rankings_big_fn'])
rank_big_df.name = rank_big_df.name.str.replace(r'\s*".+"\s*', r' ', regex=True)
rank_big_df.name = rank_big_df.name.str.strip()

In [66]:
rank_part_df = rank_big_df[['name']].drop_duplicates()

vocab_df = vocab_df.merge(rank_part_df, how='left', left_on='name_en', right_on='name')\
        .assign(check_ranks=lambda x:np.where(x['name'].notna(), True, False))\
        .rename(columns={'name':'ranks_name'})

In [67]:
rank_part_df['code'] = rank_part_df['name'].map(lambda x: doublemetaphone(x)[0])

vocab_df = vocab_df.merge(rank_part_df.groupby('code', as_index=False)['name'].agg(list).rename(columns={'name':'ranks_alt'}), on='code', how='left')


In [68]:
vocab_df['names_dif_flag'] = (vocab_df['fighters_name']==vocab_df['fights_name'])&(vocab_df['fighters_name']==vocab_df['ranks_name'])

In [69]:
for col in ['ranks_alt', 'fights_alt', 'fighters_alt']:
    vocab_df[col] = vocab_df[col].apply(lambda x: x if isinstance(x, list) else [])

vocab_df['unique_alt_flag'] = vocab_df.apply(lambda x: len(set(x['fighters_alt'] + x['fights_alt'] + x['ranks_alt']))==1 and len(x['fighters_alt'])==1 and len(x['fights_alt'])==1 and len(x['ranks_alt'])==1, axis=1)

In [70]:
vocab_df = vocab_df.assign(auto_verified = (vocab_df.filter(regex='check').all(axis=1)) & (vocab_df['unique_alt_flag']) & (vocab_df['names_dif_flag']))
bad_sel = ~vocab_df['auto_verified']

## смотрим на хорошие

## смотрим на плохие

In [72]:
vocab_df[bad_sel]

,name_rus,name_en_ds,code,manual_verified,name_en,fighters_name,check_fighters,fighters_alt,fights_name,check_fights,fights_alt,ranks_name,check_ranks,ranks_alt,names_dif_flag,unique_alt_flag,auto_verified
7,Гарри Хардвик,Harry Hardwick,HRRTK,False,Harry Hardwick,NaN,False,[],NaN,False,[],NaN,False,[],False,False,False
8,Патрисиу Фрейри,Patricio Freire,PTRSFRR,False,Patricio Freire,Patricio Freire,True,[Patricio Freire],Patricio Freire,True,[Patricio Freire],NaN,False,[],False,False,False
12,Бенуа Сен-Дени,Benoit Saint Denis,PNTSNTTNS,False,Benoit Saint Denis,Benoit Saint Denis,True,[Benoit Saint Denis],Benoit Saint Denis,True,[Benoit Saint Denis],NaN,False,[],False,False,False
18,Анте Делия,Ante Delija,ANTTLJ,False,Ante Delija,Ante Delija,True,[Ante Delija],NaN,False,[],Ante Delija,True,[Ante Delija],False,False,False
20,Роберт Рухала,Robert Ruchala,RPRTRXL,False,Robert Ruchala,Robert Ruchala,True,[Robert Ruchala],NaN,False,[],NaN,False,[],False,False,False
21,Кауэ Фернандес,Caio Fernandes,KFRNNTS,False,Kaue Fernandes,Kaue Fernandes,True,[Kaue Fernandes],Kaue Fernandes,True,[Kaue Fernandes],NaN,False,[Kauê Fernandes],False,False,False
22,Лосене Кейта,Loseine Keita,LSNKT,False,Losene Keita,Losene Keita,True,[Losene Keita],NaN,False,[],NaN,False,[],False,False,False
23,Аксель Сола,Axel Sola,AKSLSL,False,Axel Sola,NaN,False,[],NaN,False,[],NaN,False,[],False,False,False


### альтернативные

In [73]:
sel = bad_sel & (vocab_df['unique_alt_flag'])

In [74]:
vocab_df[bad_sel & (vocab_df['unique_alt_flag'])]

,name_rus,name_en_ds,code,manual_verified,name_en,fighters_name,check_fighters,fighters_alt,fights_name,check_fights,fights_alt,ranks_name,check_ranks,ranks_alt,names_dif_flag,unique_alt_flag,auto_verified


In [59]:
# можно брать из одного поля и первое, так как по unique_alt_flag только 1 значение везде
for col in ['fighters_name', 'fights_name', 'ranks_name']:
    vocab_df.loc[sel, col] = vocab_df.loc[sel, 'fights_alt'].map(lambda x: x[0]) 

In [75]:
vocab_df[bad_sel & (vocab_df['unique_alt_flag'])]

,name_rus,name_en_ds,code,manual_verified,name_en,fighters_name,check_fighters,fighters_alt,fights_name,check_fights,fights_alt,ranks_name,check_ranks,ranks_alt,names_dif_flag,unique_alt_flag,auto_verified


- гуглим и выдает текущий турнир и прошлые бои, можно проверить

In [63]:
vocab_df.loc[bad_sel & (vocab_df['unique_alt_flag']), ['auto_verified', 'manual_verified']] = True

### нет альтернатив единых

1. Если знаем, что боец точно есть пытаемся найти его под другим именем
   ```
   fights_df[fights_df.Fighter_left.str.contains('Magomedov')]
   
   ```
   Если нашли, проще пойти в начало, заменить deep seek вариант и сюда спустится (так как вариант пройдет проверки и по другим датасетам проверки )
3. Если в датасете боев нет, то удаляем, так как для заданного момента времени бессмысленно таких искать. В будущем может появится потом, по наличию в датасете боев


In [76]:
vocab_df[bad_sel & (~vocab_df['unique_alt_flag'])]

,name_rus,name_en_ds,code,manual_verified,name_en,fighters_name,check_fighters,fighters_alt,fights_name,check_fights,fights_alt,ranks_name,check_ranks,ranks_alt,names_dif_flag,unique_alt_flag,auto_verified
7,Гарри Хардвик,Harry Hardwick,HRRTK,False,Harry Hardwick,NaN,False,[],NaN,False,[],NaN,False,[],False,False,False
8,Патрисиу Фрейри,Patricio Freire,PTRSFRR,False,Patricio Freire,Patricio Freire,True,[Patricio Freire],Patricio Freire,True,[Patricio Freire],NaN,False,[],False,False,False
12,Бенуа Сен-Дени,Benoit Saint Denis,PNTSNTTNS,False,Benoit Saint Denis,Benoit Saint Denis,True,[Benoit Saint Denis],Benoit Saint Denis,True,[Benoit Saint Denis],NaN,False,[],False,False,False
18,Анте Делия,Ante Delija,ANTTLJ,False,Ante Delija,Ante Delija,True,[Ante Delija],NaN,False,[],Ante Delija,True,[Ante Delija],False,False,False
20,Роберт Рухала,Robert Ruchala,RPRTRXL,False,Robert Ruchala,Robert Ruchala,True,[Robert Ruchala],NaN,False,[],NaN,False,[],False,False,False
21,Кауэ Фернандес,Caio Fernandes,KFRNNTS,False,Kaue Fernandes,Kaue Fernandes,True,[Kaue Fernandes],Kaue Fernandes,True,[Kaue Fernandes],NaN,False,[Kauê Fernandes],False,False,False
22,Лосене Кейта,Loseine Keita,LSNKT,False,Losene Keita,Losene Keita,True,[Losene Keita],NaN,False,[],NaN,False,[],False,False,False
23,Аксель Сола,Axel Sola,AKSLSL,False,Axel Sola,NaN,False,[],NaN,False,[],NaN,False,[],False,False,False


In [77]:
# убираем согласно фильтру сверху + условие на нулевые альтернативы по fights (по пункту 2) 
vocab_df = vocab_df[~(bad_sel & (~vocab_df['unique_alt_flag']) & (vocab_df['fights_alt'].str.len()==0))]

In [78]:
vocab_df[bad_sel & (~vocab_df['unique_alt_flag'])]

C:\Users\admin\AppData\Local\Temp\ipykernel_12572\851590646.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  vocab_df[bad_sel & (~vocab_df['unique_alt_flag'])]


,name_rus,name_en_ds,code,manual_verified,name_en,fighters_name,check_fighters,fighters_alt,fights_name,check_fights,fights_alt,ranks_name,check_ranks,ranks_alt,names_dif_flag,unique_alt_flag,auto_verified
8,Патрисиу Фрейри,Patricio Freire,PTRSFRR,False,Patricio Freire,Patricio Freire,True,[Patricio Freire],Patricio Freire,True,[Patricio Freire],NaN,False,[],False,False,False
12,Бенуа Сен-Дени,Benoit Saint Denis,PNTSNTTNS,False,Benoit Saint Denis,Benoit Saint Denis,True,[Benoit Saint Denis],Benoit Saint Denis,True,[Benoit Saint Denis],NaN,False,[],False,False,False
21,Кауэ Фернандес,Caio Fernandes,KFRNNTS,False,Kaue Fernandes,Kaue Fernandes,True,[Kaue Fernandes],Kaue Fernandes,True,[Kaue Fernandes],NaN,False,[Kauê Fernandes],False,False,False


#### проверка

In [102]:
alt_name = 'Morgan Charriere'
fights_df.loc[(fights_df['Fighter_left']==alt_name)| (fights_df['Fighter_right']==alt_name), 
['Date', 'Event', 'Fighter_left', 'Fighter_right', 'Win_lose_left']].sort_values(by='Date')

,Date,Event,Fighter_left,Fighter_right,Win_lose_left
7100,2023-09-02 00:00:00,UFC Fight Night: Gane vs. Spivac,Morgan Charriere,Manolo Zecchini,W
8658,2024-04-06 00:00:00,UFC Fight Night: Allen vs. Curtis 2,Morgan Charriere,Chepe Mariscal,L
8406,2024-09-28 00:00:00,UFC Fight Night: Moicano vs. Saint Denis,Morgan Charriere,Gabriel Miranda,W
8167,2025-03-22 00:00:00,UFC Fight Night: Edwards vs. Brady,Nathaniel Wood,Morgan Charriere,W
8692,2025-07-12 00:00:00,UFC Fight Night: Lewis vs. Teixeira,Nate Landwehr,Morgan Charriere,L


In [305]:
rank_big_df[rank_big_df['name'].str.contains('Grant')]

,division_name,date,rank,name
166,light-heavyweight,2025-07-18,67,Grant Neal
410,lightweight,2025-07-18,11,Grant Dawson
628,bantamweight,2025-07-18,29,"""Dangerous"" Davey Grant"
1471,light-heavyweight,2025-07-22,72,Grant Neal
1590,middleweight,2025-07-22,91,Dwight Grant
1709,lightweight,2025-07-22,10,Grant Dawson
1928,bantamweight,2025-07-22,29,"""Dangerous"" Davey Grant"
2784,light-heavyweight,2025-07-27,84,Grant Neal
2893,middleweight,2025-07-27,94,Dwight Grant
3011,lightweight,2025-07-27,12,Grant Dawson


#### замены

In [83]:
vocab_df.loc[12, 'ranks_name'] = 'Benoit Saint Denis'

In [84]:
vocab_df[bad_sel & (~vocab_df['unique_alt_flag'])]

C:\Users\admin\AppData\Local\Temp\ipykernel_12572\851590646.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  vocab_df[bad_sel & (~vocab_df['unique_alt_flag'])]


,name_rus,name_en_ds,code,manual_verified,name_en,fighters_name,check_fighters,fighters_alt,fights_name,check_fights,fights_alt,ranks_name,check_ranks,ranks_alt,names_dif_flag,unique_alt_flag,auto_verified
8,Патрисиу Фрейри,Patricio Freire,PTRSFRR,True,Patricio Freire,Patricio Freire,True,[Patricio Freire],Patricio Freire,True,[Patricio Freire],Patricio Freire,False,[],False,False,False
12,Бенуа Сен-Дени,Benoit Saint Denis,PNTSNTTNS,True,Benoit Saint Denis,Benoit Saint Denis,True,[Benoit Saint Denis],Benoit Saint Denis,True,[Benoit Saint Denis],Benoit Saint Denis,False,[],False,False,False
21,Кауэ Фернандес,Caio Fernandes,KFRNNTS,True,Kaue Fernandes,Kaue Fernandes,True,[Kaue Fernandes],Kaue Fernandes,True,[Kaue Fernandes],Kauê Fernandes,False,[Kauê Fernandes],False,False,False


In [81]:
vocab_df.loc[[8, 12, 21], ['manual_verified']] = True

Остальные не интересны, так как на них боев не будет и предсказаний не будет

## сохранение

In [85]:
primary_cols = ['name_rus', 'fighters_name', 'fights_name', 'ranks_name', 'manual_verified', 'auto_verified']

vocab_df = vocab_df.loc[(vocab_df.auto_verified==True) | (vocab_df.manual_verified==True), primary_cols]

In [87]:

vocab_df = pd.concat([vocab_df, vocab_old_df], ignore_index=True, axis=0)

In [88]:
vocab_df.to_csv(conf['inference']['vocab_fn'], index=False)

## manual changes

In [101]:
vocab_df = pd.read_csv(conf['inference']['vocab_fn'])

### add record

In [104]:
fights_df[fights_df.Fighter_left.str.contains('desa')]

,Event,Fighter_left,Fighter_right,Win_lose_left,Win_lose_right,Method:,Round:,Time:,Time format:,Referee:,...,Distance_l_5,Distance_r_5,Clinch_l_5,Clinch_r_5,Ground_l_5,Ground_r_5,judges,judge1,judge2,judge3
4591,UFC Fight Night: Poirier vs. Gaethje,Israel Adesanya,Marvin Vettori,W,L,Decision - Split,3,5:00,3 Rnd (5-5-5),Herb Dean,...,NaN,NaN,NaN,NaN,NaN,NaN,"[('Chris Hayes', '28', '29'), ('Chris Lee', '2...","('Chris Hayes', '28', '29')","('Chris Lee', '29', '28')","('Derek Cleary', '28', '29')"
4977,UFC 234: Adesanya vs. Silva,Israel Adesanya,Anderson Silva,W,L,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Herb Dean,...,NaN,NaN,NaN,NaN,NaN,NaN,"[('Garth Harriman', '28', '29'), ('Albert Shen...","('Garth Harriman', '28', '29')","('Albert Shen', '27', '30')","('Charlie Keech', '27', '30')"
5540,UFC 248: Adesanya vs. Romero,Israel Adesanya,Yoel Romero,W,L,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),Dan Miragliotta,...,12 of 29,14 of 32,0 of 0,0 of 1,0 of 0,0 of 0,"[('Chris Lee', '46', '49'), (""Sal D'amato"", '4...","('Chris Lee', '46', '49')","(""Sal D'amato"", '47', '48')","('Ron McCarthy', '47', '48')"
5782,UFC 253: Adesanya vs. Costa,Israel Adesanya,Paulo Costa,W,L,KO/TKO,2,3:59,5 Rnd (5-5-5-5-5),Jason Herzog,...,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN
6151,UFC 263: Adesanya vs. Vettori 2,Israel Adesanya,Marvin Vettori,W,L,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),Marc Goddard,...,11 of 26,9 of 23,0 of 1,0 of 0,0 of 0,0 of 0,"[('Derek Cleary', '45', '50'), (""Sal D'amato"",...","('Derek Cleary', '45', '50')","(""Sal D'amato"", '45', '50')","('Chris Flores', '45', '50')"
6470,UFC 271: Adesanya vs. Whittaker 2,Israel Adesanya,Robert Whittaker,W,L,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),Herb Dean,...,12 of 21,8 of 17,1 of 1,2 of 2,0 of 0,0 of 0,NaN,NaN,NaN,NaN
6630,UFC 276: Adesanya vs. Cannonier,Israel Adesanya,Jared Cannonier,W,L,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),Herb Dean,...,23 of 52,18 of 24,2 of 2,2 of 2,0 of 0,0 of 0,NaN,NaN,NaN,NaN
7088,UFC 293: Adesanya vs. Strickland,Israel Adesanya,Sean Strickland,L,W,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),Marc Goddard,...,21 of 60,36 of 71,0 of 0,0 of 0,0 of 0,0 of 0,NaN,NaN,NaN,NaN
7539,UFC 281: Adesanya vs. Pereira,Israel Adesanya,Alex Pereira,L,W,KO/TKO,5,2:01,5 Rnd (5-5-5-5-5),Marc Goddard,...,7 of 13,25 of 34,2 of 2,3 of 3,0 of 0,0 of 0,NaN,NaN,NaN,NaN
7727,UFC 276: Adesanya vs. Cannonier,Israel Adesanya,Jared Cannonier,W,L,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),Herb Dean,...,23 of 52,18 of 24,2 of 2,2 of 2,0 of 0,0 of 0,NaN,NaN,NaN,NaN


In [108]:
vocab_df = pd.concat([pd.DataFrame({'name_rus':'Исраэль Адесанья', 'fighters_name':'Israel Adesanya', 
              'fights_name':'Israel Adesanya', 'ranks_name':'Israel Adesanya', 'manual_verified':True, 'auto_verified':False}, index=[0]),
           vocab_df], ignore_index=True)

### правки

### save

# Ранги русские (ufc)
Не преобразовываем, если по ним подтягивается имя, то им доверяем, иначе большим рангам

In [72]:
# что делать с русскими рейтингами
rank_df = pd.read_csv('data/inference/rank_df.csv')

In [74]:
vocab_df

,name_rus,fighters_name,fights_name,ranks_name,manual_verified,auto_verified
0,Габриэлла Фернандес,Gabriella Fernandes,Gabriella Fernandes,Gabriella Fernandes,False,True
1,Жоселин Эдвардс,Joselyne Edwards,Joselyne Edwards,Joselyne Edwards,True,True
2,Элайджа Смит,Elijah Smith,Elijah Smith,Elijah Smith,False,True
3,Джулиус Уокер,Julius Walker,Julius Walker,Julius Walker,False,True
4,Эрик Андерс,Eryk Anders,Eryk Anders,Eryk Anders,True,True
5,Майлз Джонс,Miles Johns,Miles Johns,Miles Johns,False,True
6,Андре Фили,Andre Fili,Andre Fili,Andre Fili,True,False
7,Ясмин Лусиндо,Iasmin Lucindo,Iasmin Lucindo,Iasmin Lucindo,True,True
8,Роман Долидзе,Roman Dolidze,Roman Dolidze,Roman Dolidze,False,True
10,Присцилла Кашоэйра,Priscila Cachoeira,Priscila Cachoeira,Priscila Cachoeira,False,True


In [75]:
rank_df.merge(vocab_df, left_on='name', right_on='name_rus', how='inner')

,name,rank,direction,val,division_name,date,name_rus,fighters_name,fights_name,ranks_name,manual_verified,auto_verified
0,Роман Долидзе,8,NaN,NaN,Средний вес,2025-07-08,Роман Долидзе,Roman Dolidze,Roman Dolidze,Roman Dolidze,False,True
1,Роман Долидзе,8,NaN,NaN,Средний вес,2025-07-27,Роман Долидзе,Roman Dolidze,Roman Dolidze,Roman Dolidze,False,True
2,Роман Долидзе,9,decreased,1.0,Средний вес,2025-08-04,Роман Долидзе,Roman Dolidze,Roman Dolidze,Roman Dolidze,False,True
3,Ясмин Лусиндо,8,NaN,NaN,Женский минимальный вес,2025-07-08,Ясмин Лусиндо,Iasmin Lucindo,Iasmin Lucindo,Iasmin Lucindo,True,True
4,Ясмин Лусиндо,8,NaN,NaN,Женский минимальный вес,2025-07-27,Ясмин Лусиндо,Iasmin Lucindo,Iasmin Lucindo,Iasmin Lucindo,True,True
5,Ясмин Лусиндо,8,NaN,NaN,Женский минимальный вес,2025-08-04,Ясмин Лусиндо,Iasmin Lucindo,Iasmin Lucindo,Iasmin Lucindo,True,True
6,Анджела Хилл,12,NaN,NaN,Женский минимальный вес,2025-07-08,Анджела Хилл,Angela Hill,Angela Hill,Angela Hill,False,True
7,Анджела Хилл,12,NaN,NaN,Женский минимальный вес,2025-07-27,Анджела Хилл,Angela Hill,Angela Hill,Angela Hill,False,True
8,Анджела Хилл,12,NaN,NaN,Женский минимальный вес,2025-08-04,Анджела Хилл,Angela Hill,Angela Hill,Angela Hill,False,True
